In [136]:
import pandas as pd
import os
import re
import numpy as np

#prueba
from collections import defaultdict

### Iteration 1 | Parse VCF files to table/dataframe

In [2]:
def read_vcf(file):
    with open(file, "r") as f:
        for line in f:   # Get header line
            if line.startswith('#CHROM'): hd = line; break
        hd = hd[1:].strip().split("\t")   # Remove # from CHROM and convert to list
        df = pd.read_csv(f, delimiter='\t', names=hd, comment='#')   # Save vcf as dataframe
    return df

In [3]:
#file = "/home/athena/Desktop/IiSGM-bioinfo-test/data/test.vcf"
file = "/home/athena/Desktop/IiSGM-bioinfo-test/data/10105494-0-COL2.combined.hf.SNP.final.vcf"
df = read_vcf(file)
df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,10105494-0-COL2
0,MTB_anc,1701,.,T,C,140323.33,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=0.814;DP=296;Ex...,GT:AD:DP:GQ:PL,"1/1:0,296:296:99:11020,884,0"
1,MTB_anc,2532,.,C,T,47660.33,PASS,AC=2;AF=1.00;AN=2;DP=109;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,109:109:99:3646,326,0"
2,MTB_anc,8040,.,G,A,88799.33,PASS,AC=2;AF=1.00;AN=2;DP=204;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,204:204:99:7045,612,0"
3,MTB_anc,9143,.,C,T,38105.33,PASS,AC=2;AF=1.00;AN=2;DP=87;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,87:87:99:3098,261,0"
4,MTB_anc,13460,.,G,A,33355.32,PASS,AC=2;AF=1.00;AN=2;DP=89;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,89:89:99:3127,266,0"
...,...,...,...,...,...,...,...,...,...,...
844,MTB_anc,4397324,.,G,C,22342.17,PASS,AC=2;AF=1.00;AN=2;DP=56;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,56:56:99:2155,168,0"
845,MTB_anc,4407588,.,C,T,45587.33,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=1.81;DP=108;Exc...,GT:AD:DP:GQ:PL,"1/1:0,108:108:99:3781,325,0"
846,MTB_anc,4408156,.,A,C,43801.33,PASS,AC=2;AF=1.00;AN=2;DP=88;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,88:88:99:3405,264,0"
847,MTB_anc,4408920,.,G,A,31138.27,PASS,AC=2;AF=1.00;AN=2;DP=76;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PGT:PID:PL:PS,"1|1:0,76:76:99:1|1:4408920_G_A:3471,235,0:4408920"


### Iteration 2 | Extract relevant information from parsed VCF

Remove FILTER != PASS or end col indicating heterozigois (0/1, 1/2, ...)

In [4]:
def filter_vcf(df):
    df.drop(df[df.FILTER != "PASS"].index, inplace=True)   # Drop low quality sites
    for inf in df.itertuples():   # Drop "heterozygous"
        l_line = inf[-1]
        if l_line[0] != l_line[2]: df.drop(inf[0], inplace=True)   # If the alleles are not equal
    return df

In [5]:
filter_vcf(df)

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,10105494-0-COL2
0,MTB_anc,1701,.,T,C,140323.33,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=0.814;DP=296;Ex...,GT:AD:DP:GQ:PL,"1/1:0,296:296:99:11020,884,0"
1,MTB_anc,2532,.,C,T,47660.33,PASS,AC=2;AF=1.00;AN=2;DP=109;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,109:109:99:3646,326,0"
2,MTB_anc,8040,.,G,A,88799.33,PASS,AC=2;AF=1.00;AN=2;DP=204;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,204:204:99:7045,612,0"
3,MTB_anc,9143,.,C,T,38105.33,PASS,AC=2;AF=1.00;AN=2;DP=87;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,87:87:99:3098,261,0"
4,MTB_anc,13460,.,G,A,33355.32,PASS,AC=2;AF=1.00;AN=2;DP=89;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,89:89:99:3127,266,0"
...,...,...,...,...,...,...,...,...,...,...
844,MTB_anc,4397324,.,G,C,22342.17,PASS,AC=2;AF=1.00;AN=2;DP=56;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,56:56:99:2155,168,0"
845,MTB_anc,4407588,.,C,T,45587.33,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=1.81;DP=108;Exc...,GT:AD:DP:GQ:PL,"1/1:0,108:108:99:3781,325,0"
846,MTB_anc,4408156,.,A,C,43801.33,PASS,AC=2;AF=1.00;AN=2;DP=88;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,88:88:99:3405,264,0"
847,MTB_anc,4408920,.,G,A,31138.27,PASS,AC=2;AF=1.00;AN=2;DP=76;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PGT:PID:PL:PS,"1|1:0,76:76:99:1|1:4408920_G_A:3471,235,0:4408920"


### Iteration 3 | Combine present SNP into a presence matrix

In [150]:
def presence_matrix(direct):
    dir_l = [direct + f for f in os.listdir(direct) if f.endswith(".vcf")] # Get vcf files in directory
    d = defaultdict(dict)
    for file in dir_l: # Read, filter, find mutations and create dict 
        df = read_vcf(file)
        filter_vcf(df)
        d = d_mut(df, d)
    a = pd.DataFrame(d)   # Se puede pasar a numpy sin este intermedio?
    pr_mat = np.nan_to_num(a, nan=0)   # 1/0 matrix
    return pr_mat

In [138]:
def d_mut(df, d):
    """
    dict to store mutations. id of organism as key and list of mutations (POS_REF_ALT) as value
    """
    org = df.columns[-1]   # Get sample id
    mut = list(df[["POS", "REF", "ALT"]].apply(lambda x: '_'.join(x.astype(str)), axis=1))   # Mutation id (POS_REF_ALT)
    for m in mut:   # Dictionary for storing presence of mutations
        d[org][m] = 1
    return d

In [151]:
direct = "/home/athena/Desktop/IiSGM-bioinfo-test/data/"
pr_mat = presence_matrix(direct)

In [157]:
pr_mat.shape

(859, 26)

In [115]:
# para ver cuántos snp hay
l=[]
for i in d.values():
    l = l + i

# para ver las variantes alternativas
for i in l:
    if "," in i: print(i)
        
# inserciones
for i in l:
    if re.match(".*_.+_..+", i):
        print(i)
        
# deleciones
for i in l:
    if re.match(".*_..+_.+", i):
        print(i)

954533_C_T,CACCGA
3174496_G_A,GTA
954533_C_T,CACCGA
954533_C_T,CACCGA
3174496_G_A,GTA
3174496_G_A,GTA
954533_C_T,CACCGA
3174496_G_A,GTA
954533_C_T,CACCGA
3174496_G_A,GTA
954533_C_T,CACCGA
3174496_G_A,GTA
954533_C_T,CACCGA
3174496_G_A,GTA
954533_C_T,CACCGA
3174496_G_A,GTA
954533_C_T,CACCGA
3174496_G_A,GTA
954533_C_T,CACCGA
3174496_G_A,GTA
954533_C_T,CACCGA
3174496_G_A,GTA
954533_C_T,CACCGA
3174496_G_A,GTA
3174496_G_A,GTA
954533_C_T,CACCGA
3174496_G_A,GTA
954533_C_T,CACCGA
3174496_G_A,GTA
954533_C_T,CACCGA
3174496_G_A,GTA
954533_C_T,CACCGA
2074509_C_G,*
3174496_G_A,GTA
954533_C_T,CACCGA
3174496_G_A,GTA
954533_C_T,CACCGA
3174496_G_A,GTA
954533_C_T,CACCGA
3174496_G_A,GTA
954533_C_T,CACCGA
3174496_G_A,GTA
954533_C_T,CACCGA
3174496_G_A,GTA
3174496_G_A,GTA
3174496_G_A,GTA
